In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
import keras_tuner as kt
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_excel("reporte_diario_campaña_limpio.xlsx").drop(columns='Unnamed: 0')
data = df[df["linea"]=="linea_2"]

# Ordenar por fecha
data = data.sort_values('fecha')

# Separar características y etiquetas
features = ['año', 'mes', "dia", 'dia_semana', 'state_holiday']
X = data[features]
y = data['interpolado_real_calls']


In [ ]:
def entrenamiento_gru(data,target):
    if target == 'interpolado_real_calls':
        features = ['año', 'mes', "dia", 'dia_semana', 'state_holiday']
    else: 
        features = ['año', 'mes', "dia", 'dia_semana', 'state_holiday','interpolado_real_calls']
    X = data[features]
    y = data['interpolado_real_calls']
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    def train_test_split_by_date(data, date_column, train_end_date, test_start_date):
        train_data = data[data[date_column] <= train_end_date]
        test_data = data[data[date_column] >= test_start_date]
        return train_data, test_data
    train_data, test_data = train_test_split_by_date(data, 'fecha', '2023-12-31', '2024-01-01')

    X_train, y_train = train_data[features], train_data[target]
    X_test, y_test = test_data[features], test_data[target]

    X_train_scaled = scaler_X.fit_transform(X_train)
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

    X_test_scaled = scaler_X.transform(X_test)
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

    def create_sequences(X, y, time_steps=10):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            Xs.append(X[i:(i + time_steps)])
            ys.append(y[i + time_steps])
        return np.array(Xs), np.array(ys)

    time_steps = 14
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps)
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, time_steps)
    def wape_metric(y_true, y_pred):
        return K.sum(K.abs(y_true - y_pred)) / K.sum(K.abs(y_true))
    def build_robust_gru_model(hp):
        model = Sequential()
        
        # Capas GRU
        for i in range(hp.Int('gru_layers', 1, 3)):
            model.add(GRU(units=hp.Int(f'gru_units_{i}', min_value=64, max_value=512, step=64),
                        return_sequences=(i < hp.Int('gru_layers', 1, 3) - 1),
                        recurrent_dropout=hp.Float(f'recurrent_dropout_{i}', min_value=0.0, max_value=0.3, step=0.1)))
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))
        
        # Capa de normalización por lotes
        model.add(BatchNormalization())

        # Capas densas finales
        for i in range(hp.Int('dense_layers', 1, 3)):
            model.add(Dense(units=hp.Int(f'dense_units_{i}', min_value=32, max_value=256, step=32), activation='relu'))
            model.add(Dropout(rate=hp.Float(f'dense_dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))

        model.add(Dense(1))

        # Compilación del modelo
        model.compile(optimizer=Adam(hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')),
                    loss='mse',  # Seguimos utilizando 'mse' para el entrenamiento
                    metrics=[wape_metric])  # Pero optimizamos usando WAPE
        
        return model

    tuner = kt.RandomSearch(
        build_robust_gru_model,
        objective=kt.Objective('val_wape_metric', direction='min'), 
        max_trials=15,  
        executions_per_trial=2, 
        seed = 47)

    early_stopping = EarlyStopping(
        monitor='val_wape_metric', 
        patience=7, 
        restore_best_weights=True,
        mode = "min" 
    )
    # Búsqueda de hiperparámetros
    tuner.search(
        X_train_seq, 
        y_train_seq, 
        epochs=30, 
        validation_split=0.2, 
        batch_size=32,
        callbacks=[early_stopping] )
    best_model = tuner.get_best_models(num_models=1)[0]

    best_model.fit(X_train_seq, y_train_seq, epochs=100, validation_split=0.2, batch_size=32)

    y_pred_gru_tuned = best_model.predict(X_test_seq)
    y_pred_gru_tuned_rescaled = scaler_y.inverse_transform(y_pred_gru_tuned)

    return y_pred_gru_tuned_rescaled


In [3]:
# Normalizar los datos
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

In [5]:
# Crear una función para identificar el conjunto de entrenamiento y prueba
def train_test_split_by_date(data, date_column, train_end_date, test_start_date):
    train_data = data[data[date_column] <= train_end_date]
    test_data = data[data[date_column] >= test_start_date]
    return train_data, test_data

# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split_by_date(data, 'fecha', '2023-12-31', '2024-01-01')

# Separar características y etiquetas para entrenamiento y prueba
X_train, y_train = train_data[features], train_data['interpolado_real_calls']
X_test, y_test = test_data[features], test_data['interpolado_real_calls']

In [6]:
# Escalar los datos
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [7]:
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, time_steps)


In [8]:
# Definir WAPE como métrica personalizada
def wape_metric(y_true, y_pred):
    return K.sum(K.abs(y_true - y_pred)) / K.sum(K.abs(y_true))

In [9]:

def build_robust_gru_model(hp):
    model = Sequential()
    
    # Capas GRU
    for i in range(hp.Int('gru_layers', 1, 3)):
        model.add(GRU(units=hp.Int(f'gru_units_{i}', min_value=64, max_value=512, step=64),
                      return_sequences=(i < hp.Int('gru_layers', 1, 3) - 1),
                      recurrent_dropout=hp.Float(f'recurrent_dropout_{i}', min_value=0.0, max_value=0.3, step=0.1)))
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))
    
    # Capa de normalización por lotes
    model.add(BatchNormalization())

    # Capas densas finales
    for i in range(hp.Int('dense_layers', 1, 3)):
        model.add(Dense(units=hp.Int(f'dense_units_{i}', min_value=32, max_value=256, step=32), activation='relu'))
        model.add(Dropout(rate=hp.Float(f'dense_dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(1))

    # Compilación del modelo
    model.compile(optimizer=Adam(hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')),
                  loss='mse',  # Seguimos utilizando 'mse' para el entrenamiento
                  metrics=[wape_metric])  # Pero optimizamos usando WAPE
    
    return model


In [13]:
# Definición del tuner para minimizar WAPE
tuner = kt.RandomSearch(
    build_robust_gru_model,
    objective=kt.Objective('val_wape_metric', direction='min'), 
    max_trials=15,  
    executions_per_trial=2, 
    seed = 47)

early_stopping = EarlyStopping(
    monitor='val_wape_metric', 
    patience=7, 
    restore_best_weights=True,
    mode = "min" 
)
# Búsqueda de hiperparámetros
tuner.search(
    X_train_seq, 
    y_train_seq, 
    epochs=30, 
    validation_split=0.2, 
    batch_size=32,
    callbacks=[early_stopping]  # Agregamos el callback de Early Stopping
)

Trial 15 Complete [00h 04m 43s]
val_wape_metric: 0.39876797795295715

Best val_wape_metric So Far: 0.3517134338617325
Total elapsed time: 00h 46m 20s


In [14]:
# Obtener el mejor modelo basado en WAPE
best_model = tuner.get_best_models(num_models=1)[0]

# Entrenamiento adicional del mejor modelo si es necesario
best_model.fit(X_train_seq, y_train_seq, epochs=100, validation_split=0.2, batch_size=32)

# Evaluación del mejor modelo
y_pred_gru_tuned = best_model.predict(X_test_seq)
y_pred_gru_tuned_rescaled = scaler_y.inverse_transform(y_pred_gru_tuned)



Epoch 1/100


c:\Users\LauraVanessaRocaOroz\Desktop\TFM\tfm_venv\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 36 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


36/36 ━━━━━━━━━━━━━━━━━━━━ 23s 164ms/step - loss: 0.1962 - wape_metric: 0.3323 - val_loss: 0.2176 - val_wape_metric: 0.5909
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 128ms/step - loss: 0.2095 - wape_metric: 0.3325 - val_loss: 0.1492 - val_wape_metric: 0.4393
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - loss: 0.2009 - wape_metric: 0.3337 - val_loss: 0.1114 - val_wape_metric: 0.3904
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 113ms/step - loss: 0.1950 - wape_metric: 0.3417 - val_loss: 0.2064 - val_wape_metric: 0.5936
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - loss: 0.1920 - wape_metric: 0.3140 - val_loss: 0.1693 - val_wape_metric: 0.5124
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 113ms/step - loss: 0.1729 - wape_metric: 0.3094 - val_loss: 0.1269 - val_wape_metric: 0.4231
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 142ms/step - loss: 0.1843 - wape_metric: 0.3290 - val_loss: 0.1077 - val_wape_metric: 0.3786
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 100ms/step - loss: 0.180

ValueError: Found array with dim 3. None expected <= 2.

In [15]:
y_test_rescaled= scaler_y.inverse_transform(y_test_seq)

# Cálculo de WAPE para el modelo ajustado
wape_gru_tuned = wape_metric(y_test_rescaled, y_pred_gru_tuned_rescaled)

# Mostrar el resultado de WAPE
print(f"WAPE GRU Tuned: {wape_gru_tuned:.2f}%")

WAPE GRU Tuned: 0.13%
